In [33]:
from t1t2_mwpm import Setup, DecoherenceModel, add_noise, get_matching
from t1t2_mwpm.circuits.rep_code import init_qubits, qec_round, log_meas


from stim import Circuit

In [2]:
relax_times = [10000, 20000, 30000, 40000, 50000]
deph_times = [10000, 20000, 30000, 40000, 50000]
gate_durations = dict(X=100, H=100, CZ=200, M=1000, R=600)

In [3]:
setup = Setup(relax_times, deph_times, gate_durations)
model = DecoherenceModel(setup)

In [4]:
data_inds = [0, 2, 4]
anc_inds = [1, 3]

qubit_inds = set(data_inds + anc_inds)

In [5]:
data_init = [0, 0, 0]
init_circ = init_qubits(model, data_inds, anc_inds, data_init=data_init)
qec_circ = qec_round(model, data_inds, anc_inds)
meas_circ = log_meas(model, data_inds, anc_inds, comp_rounds=1)

constructed_circ = init_circ + qec_circ + meas_circ

In [23]:
constructed_circ.without_noise()

stim.Circuit('''
    R 0 2 4 1 3
    TICK
    H 1 3
    TICK
    CZ 0 1 2 3
    TICK
    CZ 1 2 3 4
    TICK
    H 1 3
    TICK
    M 1 3
    DETECTOR rec[-2]
    DETECTOR rec[-1]
    TICK
    R 1 3
    TICK
    M 0 2 4
    DETECTOR rec[-2] rec[-3] rec[-5]
    DETECTOR rec[-1] rec[-2] rec[-4]
    OBSERVABLE_INCLUDE(0) rec[-3] rec[-2] rec[-1]
    TICK
''')

In [24]:
input_circuit = Circuit('''
    R 0 2 4 1 3
    TICK
    H 1 3
    TICK
    CZ 0 1 2 3
    TICK
    CZ 1 2 3 4
    TICK
    H 1 3
    TICK
    M 1 3
    DETECTOR rec[-2]
    DETECTOR rec[-1]
    TICK
    R 1 3
    TICK
    M 0 2 4
    DETECTOR rec[-2] rec[-3] rec[-5]
    DETECTOR rec[-1] rec[-2] rec[-4]
    OBSERVABLE_INCLUDE(0) rec[-3] rec[-2] rec[-1]
    TICK
''')

In [25]:
noisy_circuit = add_noise(input_circuit, model)

In [26]:
assert noisy_circuit.without_noise() == input_circuit

In [37]:
matching = get_matching(noisy_circuit)